In [33]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [34]:
v = '5-1'

In [35]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.add_probs'
c_model = 'deberta-v3-small.flag-context4-2-d-oof.add_probs'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [36]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [37]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval.160_30/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

In [38]:
# d = pd.DataFrame(gezi.batch2list(xc))
# dg = d.groupby('pred_id')
# max_preds = dg['pred'].max()
# min_preds = dg['pred'].min()
# max_cls_preds = dg['cls_pred'].max()
# min_cls_preds = dg['cls_pred'].min()

In [39]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
# xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval.160_30/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

ic(calc_metric(xp, 'cls_pred'))
ic(calc_metric(xc, 'reg_pred'))
ic(calc_metric(xc, 'cls_pred'))
ic(calc_metric(xpc, 'cls_pred'))

[08/09/22 21:26:31] 73525872.py:9 in <module>
                    calc_metric(xp, 'cls_pred'): 0.8940525259130645
[08/09/22 21:26:39] 73525872.py:10 in <module>
                    calc_metric(xc, 'reg_pred'): 0.9120765273482367
[08/09/22 21:26:46] 73525872.py:11 in <module>
                    calc_metric(xc, 'cls_pred'): 0.9132059636520105
[08/09/22 21:26:54] 73525872.py:12 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9011719210628208


0.9011719210628208

In [40]:
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  # if prob.max() < 10:
  if prob.max() < 0.8:
    marks.append(1)
    preds.append(pc_preds.get(cid, pred))
    probs.append(pc_probs.get(cid, prob))
    cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  else:
    marks.append(0)
    preds.append(pred)
    probs.append(prob)
    cls_preds.append(cls_pred)
    
xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())
ic(calc_metric(xpc, 'cls_pred'))

[08/09/22 21:27:02] 2337548337.py:27 in <module>
                    xpc['mark'].mean(): 0.5019261406823974
[08/09/22 21:27:09] 2337548337.py:28 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9020743859652581


0.9020743859652581

In [41]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=10):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  # x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  # x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
        # x[f'top_rank_{i}'] = idxes[i]
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  # diff_feats(x, 'cls_rank_pred', 'rank_pred', 'cls_rank')
  return x

def gen_cfeat(x_):
  topn = 10
  x = {}
  keys = [
    # 'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    ]
  x = {k: x_[k] for k in keys}
  # x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  # x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  # x['reg_rank_pred'] = x['reg_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
        # x[f'tpop_rank_{i}']  = idxes[i] 
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  # diff_feats(x, 'cls_rank_pred', 'reg_rank_pred', 'cls_reg_rank')
  # diff_feats(x, 'cls_rank_pred', 'cls2_rank_pred', 'cls_cls2_rank')
  # diff_feats(x, 'cls2_rank_pred', 'reg_rank_pred', 'cls2_reg_rank')
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]

  feats = []
  for i in tqdm(range(len(xs)), desc='merge_feats'):
    fe1 = p_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')
    
    # diff_feats(fe, fe1['rank_pred'], fec['reg_rank_pred'], 'pc_cls_rank')
    # diff_feats(fe, fe1['cls_rank_pred'], fec['reg_rank_pred'], 'pc_cls_reg_rank')
    # diff_feats(fe, fepc['rank_pred'], fec['reg_rank_pred'], 'pcc_cls_rank')
    # diff_feats(fe, fepc['cls_rank_pred'], fec['reg_rank_pred'], 'pcc_cls_reg_rank')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)

    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [42]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [43]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [44]:
gezi.set_fold(df, 5, 'ancestor_id')

In [45]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

135

In [46]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [47]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [48]:
from catboost import CatBoostRegressor, CatBoostRanker, CatBoostClassifier
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              # 'reg_lambda': 7.960622217848342e-07, 
              'l2_leaf_reg': 3,
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              # 'max_depth': 10, 
              # 'max_depth': 8,
              'max_depth': 6,
              # 'max_depth': 5,
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [49]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
ic(cbt_score)

0:	learn: 0.2523564	test: 0.2523564	test1: 0.2522917	best: 0.2522917 (0)	total: 55.2ms	remaining: 9m 12s
100:	learn: 0.0662093	test: 0.0662093	test1: 0.0673568	best: 0.0673568 (100)	total: 4.06s	remaining: 6m 38s
200:	learn: 0.0595803	test: 0.0595803	test1: 0.0608971	best: 0.0608971 (200)	total: 7.81s	remaining: 6m 20s
300:	learn: 0.0587525	test: 0.0587525	test1: 0.0601223	best: 0.0601223 (300)	total: 11.7s	remaining: 6m 17s
400:	learn: 0.0579752	test: 0.0579752	test1: 0.0593766	best: 0.0593766 (400)	total: 15.8s	remaining: 6m 17s
500:	learn: 0.0573788	test: 0.0573788	test1: 0.0588286	best: 0.0588286 (500)	total: 19.8s	remaining: 6m 16s
600:	learn: 0.0570106	test: 0.0570106	test1: 0.0584960	best: 0.0584960 (600)	total: 24.1s	remaining: 6m 17s
700:	learn: 0.0567383	test: 0.0567383	test1: 0.0582589	best: 0.0582589 (700)	total: 28.1s	remaining: 6m 13s
800:	learn: 0.0565312	test: 0.0565312	test1: 0.0580850	best: 0.0580850 (800)	total: 32.1s	remaining: 6m 8s
900:	learn: 0.0563718	test: 0.05

[08/09/22 21:34:44] 3198672829.py:8 in <module>
                    cbt_score: 0.9197607411037639


0.9197607411037639

In [50]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [51]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          # 'num_leaves': 32,
          # 'num_leaves': 24,
          'num_leaves': 12,
          'min_data_in_leaf': 300,
          # 'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                20000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=200)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.082803 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32022
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 135
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.067215	valid_1's l1: 0.0682808
[200]	training's l1: 0.0600246	valid_1's l1: 0.0612298
[300]	training's l1: 0.0589993	valid_1's l1: 0.0602554
[400]	training's l1: 0.0580944	valid_1's l1: 0.0594058
[500]	training's l1: 0.0575162	valid_1's l1: 0.0588837
[600]	training's l1: 0.0572022	valid_1's l1: 0.0585898
[700]	training's l1: 0.056864	valid_1's l1: 0.058295
[800]	training's l1: 0.0566528	valid_1's l1: 0.0581156
[900]	training's l1: 0.0564973	valid_1's l1: 0.0579858
[1000]	training's l1: 0.0563807	valid_1's l1: 0.0578969
[1100]	training's l1: 0.0562888	valid_1's 

[08/09/22 21:38:34] 155394547.py:37 in <module>
                    lgb_score: 0.9196994418536476


0.9196994418536476

In [52]:
# model = lgb.LGBMRanker(
#     objective="lambdarank",
#     metric="ndcg",
#     boosting_type="dart",
#     n_estimators=100,
#     importance_type='gain',
#     verbose=10,
#     random_state = 17
# )

# qids_train = dtrain.groupby(['id'])['cell_id'].count().values
# # y_train2 = [int(x * 1000) for x in y_train]

# model.fit(
#     X=X_train,
#     y=y_train2,
#     group=qids_train,
# )

In [53]:
# import xgboost as xgb
# params = {
#       'reg_lambda': 7.960622217848342e-07, 
#                       'subsample': 0.7422597612762745,
#                       # 'bagging_temperature': 0.2,
#                       'max_depth': 10, 
#                       # 'early_stopping_rounds': 500,
#                       # 'n_estimators': 10000,
#                       # 'cat_features': [],
#                       # 'loss_function': 'MAE',
#                       'min_child_samples': 100,
#                       # eval_metric='MAE',
# }
# model = xgb.XGBRanker(booster="gbtree", 
#                       # objective="rank:ndcg",
#                       objective='rank:pairwise',
#                       tree_method="gpu_hist", 
#                       sampling_method="gradient_based",
#                       n_estimators=5000,
#                       learning_rate=0.03,
#                       **params
#                       )
# train_groups = dtrain.groupby('id').size().to_numpy()
# eval_groups = dvalid.groupby('id').size().to_numpy()
# model.fit(X_train, 
#           y_train, 
#           group=train_groups, 
#           eval_set=[(X_train, y_train),(X_valid, y_valid)],
#           eval_group=[train_groups, eval_groups],
#           verbose=100)


In [54]:
# xgb_model = model

In [55]:
# dvalid['xgb_pred'] = xgb_model.predict(dvalid[cols])
# dvalid['xgb_pred'] = [gezi.sigmoid(x) for x in dvalid.xgb_pred.values]
# xgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.xgb_pred.values})
# ic(xgb_score)

In [56]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [57]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir(f'../working/trees{v}')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees{v}/{fold}.cbt')
  dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    20000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=200)
  lgb_model.save_model(f'../working/trees{v}/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cbt_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/09/22 21:38:53] 1820686574.py:21 in <module>
                    cbt_score: 0.9197607411037639
[08/09/22 21:39:14] 1820686574.py:36 in <module>
                    lgb_score: 0.9196994418536476
[08/09/22 21:39:16] 1820686574.py:38 in <module>
                    score: 0.9199690334536488
[08/09/22 21:39:16] 1820686574.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9197607411037639
                    np.asarray(lgb_scores).mean(): 0.9196994418536476
                    np.asarray(scores).mean(): 0.9199690334536488


0:	learn: 0.2523919	test: 0.2523919	test1: 0.2523018	best: 0.2523018 (0)	total: 55.9ms	remaining: 9m 19s
500:	learn: 0.0577519	test: 0.0577519	test1: 0.0573472	best: 0.0573472 (500)	total: 19.7s	remaining: 6m 13s
1000:	learn: 0.0565768	test: 0.0565768	test1: 0.0563869	best: 0.0563869 (1000)	total: 39.5s	remaining: 5m 55s
1500:	learn: 0.0561109	test: 0.0561109	test1: 0.0560844	best: 0.0560844 (1500)	total: 59.4s	remaining: 5m 36s
2000:	learn: 0.0557446	test: 0.0557446	test1: 0.0559183	best: 0.0559161 (1998)	total: 1m 19s	remaining: 5m 16s
2500:	learn: 0.0554194	test: 0.0554194	test1: 0.0558103	best: 0.0558103 (2500)	total: 1m 38s	remaining: 4m 55s
3000:	learn: 0.0551392	test: 0.0551392	test1: 0.0557366	best: 0.0557348 (2986)	total: 1m 58s	remaining: 4m 35s
3500:	learn: 0.0548832	test: 0.0548832	test1: 0.0556856	best: 0.0556856 (3498)	total: 2m 17s	remaining: 4m 15s
4000:	learn: 0.0546677	test: 0.0546677	test1: 0.0556440	best: 0.0556440 (4000)	total: 2m 37s	remaining: 3m 56s
4500:	learn:

[08/09/22 21:45:56] 1820686574.py:21 in <module>
                    cbt_score: 0.9219768639917801


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.063491 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32020
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 135
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0578745	valid_1's l1: 0.0574244
[1000]	training's l1: 0.0568113	valid_1's l1: 0.0564942
[1500]	training's l1: 0.0564607	valid_1's l1: 0.0562438
[2000]	training's l1: 0.0561464	valid_1's l1: 0.056085
[2500]	training's l1: 0.0559586	valid_1's l1: 0.0559732
[3000]	training's l1: 0.0557954	valid_1's l1: 0.0558902
[3500]	training's l1: 0.0556637	valid_1's l1: 0.0558382
[4000]	training's l1: 0.055539	valid_1's l1: 0.0558035
[4500]	training's l1: 0.0554613	valid_1's l1: 0.0557766
[5000]	training's l1: 0.0553675	valid_1's l1: 0.0557528
[5500]	training's l1: 0.0552767	v

...

[08/09/22 22:09:14] 1820686574.py:36 in <module>
                    lgb_score: 0.9239861469026502
[08/09/22 22:09:16] 1820686574.py:38 in <module>
                    score: 0.9243274701898944
[08/09/22 22:09:16] 1820686574.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9219095475512511
                    np.asarray(lgb_scores).mean(): 0.9217376431911233
                    np.asarray(scores).mean(): 0.9220406377384486


0:	learn: 0.2523247	test: 0.2523247	test1: 0.2524100	best: 0.2524100 (0)	total: 56.8ms	remaining: 9m 28s
500:	learn: 0.0575632	test: 0.0575632	test1: 0.0587663	best: 0.0587663 (500)	total: 20.9s	remaining: 6m 35s
1000:	learn: 0.0563552	test: 0.0563552	test1: 0.0577415	best: 0.0577415 (1000)	total: 42.4s	remaining: 6m 20s
1500:	learn: 0.0558500	test: 0.0558500	test1: 0.0573966	best: 0.0573966 (1500)	total: 1m 3s	remaining: 6m 2s
2000:	learn: 0.0554673	test: 0.0554673	test1: 0.0572104	best: 0.0572104 (2000)	total: 1m 25s	remaining: 5m 40s
2500:	learn: 0.0551661	test: 0.0551661	test1: 0.0570749	best: 0.0570749 (2500)	total: 1m 46s	remaining: 5m 18s
3000:	learn: 0.0548843	test: 0.0548843	test1: 0.0569870	best: 0.0569870 (3000)	total: 2m 7s	remaining: 4m 57s
3500:	learn: 0.0546430	test: 0.0546430	test1: 0.0569292	best: 0.0569289 (3494)	total: 2m 28s	remaining: 4m 36s
4000:	learn: 0.0544121	test: 0.0544121	test1: 0.0568922	best: 0.0568916 (3994)	total: 2m 50s	remaining: 4m 15s
4500:	learn: 0

[08/09/22 22:16:21] 1820686574.py:21 in <module>
                    cbt_score: 0.9150715113908952


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.065852 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32021
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 135
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0575753	valid_1's l1: 0.058692
[1000]	training's l1: 0.0564634	valid_1's l1: 0.057722
[1500]	training's l1: 0.0561074	valid_1's l1: 0.057449
[2000]	training's l1: 0.0559081	valid_1's l1: 0.0573202
[2500]	training's l1: 0.0557462	valid_1's l1: 0.0572263
[3000]	training's l1: 0.055639	valid_1's l1: 0.0571696
[3500]	training's l1: 0.0555436	valid_1's l1: 0.0571273
[4000]	training's l1: 0.0554294	valid_1's l1: 0.0570823
[4500]	training's l1: 0.0553356	valid_1's l1: 0.057043
[5000]	training's l1: 0.0552266	valid_1's l1: 0.0570025
[5500]	training's l1: 0.0551601	vali

[08/09/22 22:20:52] 1820686574.py:36 in <module>
                    lgb_score: 0.9149698974038174
[08/09/22 22:20:55] 1820686574.py:38 in <module>
                    score: 0.9152534967858554
[08/09/22 22:20:55] 1820686574.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9205419403191799
                    np.asarray(lgb_scores).mean(): 0.9203840940336621
                    np.asarray(scores).mean(): 0.9206832095479299


In [58]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614